In [125]:
!pip install bert-extractive-summarizer
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q tensorflow==2.1

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np

from pathlib import Path

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


--------------------------------------------------

DATA LOADER

In [127]:
check_path = "/content/train.csv"
my_file = Path(check_path)
flag = 0
if flag and not my_file.is_file():
    check_path = "/content/AMICorpusXML/data/ami-summary/abstractive/ES2004a.abssumm.txt"

    my_file = Path(check_path)
    if not my_file.is_file():
        if not Path("/content/AMICorpusXML/data").is_dir():
            ! git clone https://github.com/gcunhase/AMICorpusXML
        ! python /content/AMICorpusXML/main_obtain_meeting2summary_data.py --summary_type abstractive
    else:
        print("Data already prepared... Importing\n")

    val = "ES2003, ES2011, IS1008, TS3004, TS3006".split(',')
    train = """ES2002, ES2005, ES2006, ES2007, ES2008, ES2009, ES2010, ES2012, ES2013, ES2015, 
                ES2016, IS1000, IS1001, IS1003, IS1004, IS1006, IS1007, TS3005, TS3008, TS3009, TS3010, 
                TS3011, TS3012""".split(",")
    test = "ES2004, ES2014, IS1009, TS3003, TS3007".split(",")

    story_directory = r'/content/AMICorpusXML/data/ami-transcripts-stories/abstractive'
    sum_directory = r'/content/AMICorpusXML/data/ami-summary/abstractive'

    story_val = []
    for filename in os.listdir(story_directory):
        if filename.endswith(".story"):
            for each in val:
                if each.strip() in str(filename):
                    story_val.append(filename)

    def data_read(f1):
        with open(f1, 'r') as file:
            data1 = file.read().replace('\n', '')
        return data1

    val_data = []
    val_summ = []
    for each in story_val:
        a = each.split('.')[0]
        story_file = story_directory+'/'+each
        summ_file = sum_directory+'/'+a+".abssumm.txt"
        data = data_read(story_file)
        val_data.append(data)
        data = data_read(summ_file)
        val_summ.append(data)
else:
    train = pd.read_csv("./content/train.csv", sep='\t')
    train = train.drop(['Unnamed: 0'],axis=1)
    test = pd.read_csv("./content/test.csv", sep='\t')
    test = test.drop(['Unnamed: 0'],axis=1)
    val = pd.read_csv("./content/val.csv", sep='\t')
    val = val.drop(['Unnamed: 0'],axis=1)

In [128]:
train.head()

,inputs,targets
0,'Kay. I'll go first. No Nathan's fine. It's ei...,The Industrial Designer presented an analysis ...
1,Um I'm Craig and I'm User Interface. Yeah. Wel...,The project manager introduced the upcoming pr...
2,How was lunch? Mm. Why? Mm-hmm. W we can use s...,The project manager stated the agenda and set ...
3,"Uh we don't have any changes, do we? So no. It...",The UI and ID presented a prototype drawing of...
4,Hold that. Okay. Okay. Mm. Mm-hmm. Mm-hmm. Mm....,The meeting begins with the group trying to re...


----------------------

## EXTRACTIVE SUMMARY GENERATOR

BERTSUM

In [129]:
from summarizer import Summarizer
from tqdm import tqdm
model = Summarizer()

In [130]:
test = test[:4]

In [131]:
ext_summary = []

for index, row in tqdm(test.iterrows()):
    input = row['inputs']
    target = row['targets']
    print(input)
    result = model(input, max_length=70, num_sentences=2)
    summ = "".join(result)
    ext_summary.append(summ)

test['ext_summary'] = ext_summary



0it [00:00, ?it/s]

Thanks. Yeah, that's probably a better one, to discuss it straight away. Um I have a lot of the information there. It might not be very clear. Unless you want to plug it back in to yours. I think the scroll wheel um Yeah, and if we're going for sleek and sexy, I think a scroll wheel is maybe a bit kinda bulky? I I've got um pictures well I've seen pictures with it kind of sticking off the side of it, and they don't really look great. Yeah, a simple pushbuttons. Hmm. But But the way that I interpret that um it doesn't seem to send out a signal to the telly, it just it's like a parrot just rep reply replying to your message. So I don't think it would effect our circuit board. Yeah. And I don't think you could really perform any of the remote functions with it. 'Cause the example that they've given there is good morning coffee machine, good morning Jo. It might be useful to say like where are you remote. Here I am, Jo. But I think that's maybe as far as that one could go? Yeah, just as a 



1it [00:30, 30.46s/it]

Good afternoon. So Hello. No problem. Okay. That's possible on uh this time of day. Starts at three o'clock. Uh Uh, let me see. Our fourth and last meeting. There he is. Yes. Okay this our last meeting. In this meeting we will discuss our final design. And we will do some evaluation about the, not only the product, but also the project. And then we're going to close the project today as well. So after this you will be uh free to go and uh spend all your money. And this uh design, detailed design meeting uh we will discuss uh the look-and-feel of the design, the usability interface design, and we will do the product evaluation. Um, in order to do that we have this agenda. We'll have the prototype presentation first. Then we will set up some evaluation cri um criteria. Then we will look at the finance. Uh we will have to see if everything we wanna do is also possible within our budget. Because everything costs money, and the more functions you wanna have the more money it will cost. So w



2it [01:18, 35.71s/it]

Okay. Everybody found his place again? Yeah? That's nice. Okay so this is our second meeting. And uh still failing? Uh now we're going um into the functional design. Um important thing of this phase is that we're going to uh try to get an agreement about the user requirements, technical function design, and the working design. So that we can move onto the second uh phase. But first this phase. Um first an announcement. There's a little adaptation in the air conditioning system. So There's our ghost mouse again. That that means that you can have a little trouble with, little trouble with the air conditioning, that's because of this uh It's in wing C_ and E_. So it should be over in a in a while, couple of days. But it's going to be cold anyway, so I don't think you're gonna need it. Then our agenda. Now first the opening. Uh this time I will take the minutes. Uh you're going to have a presentation. All of you. Um and we've got forty minutes for the whole uh prese for the whole uh presen



3it [01:52, 35.30s/it]

Hi. Oh. Good morning. I'm Juergen Toffs. I'm the User Interface Designer. Ach. Ooph. Very artistic. Okay. Wow. He does have body uh Huh. You don't have to change the colour and the pen width during uh the drawing. Or You have to draw uh push hard on the pen or uh Oh. Uh hmm. Okay. Oh. It's a very bloody cat. It's a frog. Uh it's an apple. A dog? Huh? Oh, it is a turtle. It's uh inside its shell. You'll be uh finished sooner. Wow. Yes. Okay. Wha Thank you. Yeah. Um How did it work? Performance? And a pen? Um Well, very good. I just uh thought I'd pick the easiest one. Well, we had a dog, a few years ago. And and it, yeah, when it died we didn't get a new one or something. But uh Uh a Graphical User Designer, so Hey. Oh. No. Can work together. Ah colour. Well, it's okay. Yeah. Uh-oh. Oh, he is the artistic design. Remote control animal. Well with the interface, it might be easier to ha to draw here and display there uh. Than on the, with Yeah, I m I mean like uh like on here, drawing dra



4it [02:22, 35.55s/it]


In [ ]:
test.to_csv('test_ext.csv', sep='\t')
test.head()

,inputs,targets,ext_summary
0,"Thanks. Yeah, that's probably a better one, to...",The Project Manager reviewed the minutes from ...,"Yeah, that's probably a better one, to discuss..."
1,Good afternoon. So Hello. No problem. Okay. Th...,In the detailed design meeting the team create...,In this meeting we will discuss our final desi...
2,Okay. Everybody found his place again? Yeah? T...,The project manager stated the agenda and the ...,Uh now we're going um into the functional desi...
3,Hi. Oh. Good morning. I'm Juergen Toffs. I'm t...,The participants introduced themselves and the...,Or You have to draw uh push hard on the pen or...
4,"Morning. Yep. My name's Frank. Thank you. Hmm,...",The project manager introduced himself to the ...,"Can't say I paid much attention to it, but Oh ..."


## EXTRACTIVE TO ABSTRACTIVE

GPT2 Decoder

In [132]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
modelgpt2 = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
abs_summary = []

for index, row in tqdm(test.iterrows()):
    input = row['ext_summary']
    print(input)
    input_ids = tokenizer.encode(input, return_tensors='tf')
    summ = modelgpt2.generate(input_ids, max_length=300)
    summ_decoded = tokenizer.decode(summ[0], skip_special_tokens=True)
    abs_summary.append(summ_decoded)

test['abs_summary'] = abs_summary



0it [00:00, ?it/s]

Yeah, that's probably a better one, to discuss it straight away. And it says that I think it said the cost of that isn't too much. Maybe the spongy feel is something we could think about, um.




1it [00:51, 51.84s/it]

In this meeting we will discuss our final design. Were the means sufficient for the tasks uh we had to do? You can you can change markets where by changing the front covers.




2it [01:43, 51.82s/it]

...



18it [15:32, 51.69s/it]

It's twelve point f Another question is like sorry. Um, findings, I've got my nice little picture there. Be interesting to see what our design people make of that.




19it [16:23, 51.59s/it]

Okay, so we've got uh so we've got new project requirements. I'll just um I'll grab the wire out the back of this one. So with that we'd kind of by-pass any problems with Yeah.




20it [17:15, 51.78s/it]


BEAM

In [ ]:
abs_summary = []

for index, row in tqdm(test.iterrows()):
    input = row['ext_summary']
    print(input)
    input_ids = tokenizer.encode(input, return_tensors='tf')

    beam_output = modelgpt2.generate(
        input_ids, 
        max_length=300, 
        num_beams=5, 
        early_stopping=True
    )

    summ_decoded =tokenizer.decode(beam_output[0], skip_special_tokens=True)
    abs_summary.append(summ_decoded)

test['beam_abs_summary'] = abs_summary



0it [00:00, ?it/s]

Yeah, that's probably a better one, to discuss it straight away. And it says that I think it said the cost of that isn't too much. Maybe the spongy feel is something we could think about, um.




1it [02:39, 159.13s/it]

In this meeting we will discuss our final design. Were the means sufficient for the tasks uh we had to do? You can you can change markets where by changing the front covers.




2it [05:19, 159.56s/it]

...

And my role is to design uh a television remote control. Okay, I think I have enough to think about 'til our next meeting. Now, of course, the other thing to think there is maybe the design.




13it [34:36, 159.51s/it]

Um, once again I'm uh gonna take minutes. Because I think it will Oh, five minutes from to finish meeting. Yeah, the they are big and clear.. Yeah, but but the teletext button.




14it [37:15, 159.35s/it]

It wo it won't shout out I'm here or something. There are innovations in that that people don't really want. I think the agenda one was where the um price was, wasn't it?


In [ ]:
test.head()

1051

TOP P

In [135]:
abs_summary = []

for index, row in tqdm(test.iterrows()):
    input = row['ext_summary']
    print(input)
    input_ids = tokenizer.encode(input, return_tensors='tf')

    # set seed to reproduce results. Feel free to change the seed though to get different results
    tf.random.set_seed(0)
    # deactivate top_k sampling and sample only from 92% most likely words
    sample_output = modelgpt2.generate(
        input_ids, 
        do_sample=True, 
        max_length=300, 
        top_p=0.92, 
        top_k=0
    )
    summ_decoded = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    abs_summary.append(summ_decoded)
test['top_P_abs_summary'] = abs_summary



0it [00:00, ?it/s]

Yeah, that's probably a better one, to discuss it straight away. And it says that I think it said the cost of that isn't too much. Maybe the spongy feel is something we could think about, um.




1it [00:54, 54.29s/it]

In this meeting we will discuss our final design. Were the means sufficient for the tasks uh we had to do? You can you can change markets where by changing the front covers.




2it [01:27, 47.93s/it]

Uh now we're going um into the functional design. Um this is uh a bit pity for the Marketing uh Expert.




3it [02:25, 51.13s/it]

Or You have to draw uh push hard on the pen or uh Oh. And they have uh Yeah, that's uh very important uh for them. But if you say uh is it uh uh one way or multi-purpose, that's a Why?




4it [03:19, 49.92s/it]


In [136]:
test.head()

,inputs,targets,ext_summary,top_P_abs_summary
0,"Thanks. Yeah, that's probably a better one, to...",The Project Manager reviewed the minutes from ...,"Yeah, that's probably a better one, to discuss...","Yeah, that's probably a better one, to discuss..."
1,Good afternoon. So Hello. No problem. Okay. Th...,In the detailed design meeting the team create...,In this meeting we will discuss our final desi...,In this meeting we will discuss our final desi...
2,Okay. Everybody found his place again? Yeah? T...,The project manager stated the agenda and the ...,Uh now we're going um into the functional desi...,Uh now we're going um into the functional desi...
3,Hi. Oh. Good morning. I'm Juergen Toffs. I'm t...,The participants introduced themselves and the...,Or You have to draw uh push hard on the pen or...,Or You have to draw uh push hard on the pen or...


ROUGE

In [137]:
!pip install rouge-score
from rouge_score import rouge_scorer

In [139]:
def rogue_sc(b, a):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
    scores = scorer.score(b,a)
    # print(scores)
    return scores

In [141]:
import numpy as np

rouge1 = []
rouge2 = []
# word_capt = []
for index, row in test.iterrows():
    rouge1.append(rogue_sc(row['targets'], row['top_P_abs_summary'])['rouge1'][1])
    rouge2.append(rogue_sc(row['targets'], row['top_P_abs_summary'])['rouge2'][1])
    # word_capt.append(rogue_sc(row['targets'], row['Summary annotated'])['rouge1'][1])
print("------ top_P_abs_summary-------")
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))

for index, row in test.iterrows():
    rouge1.append(rogue_sc(row['targets'], row['ext_summary'])['rouge1'][1])
    rouge2.append(rogue_sc(row['targets'], row['ext_summary'])['rouge2'][1])
    # word_capt.append(rogue_sc(row['targets'], row['Summary annotated'])['rouge1'][1])
print("------ Extractive_summary-------")
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))


for index, row in test.iterrows():
    rouge1.append(rogue_sc(row['targets'], row['beam_abs_summary'])['rouge1'][1])
    rouge2.append(rogue_sc(row['targets'], row['beam_abs_summary'])['rouge2'][1])
    # word_capt.append(rogue_sc(row['targets'], row['Summary annotated'])['rouge1'][1])
print("------ Beams_summary-------")
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))


for index, row in test.iterrows():
    rouge1.append(rogue_sc(row['targets'], row['abs_summary'])['rouge1'][1])
    rouge2.append(rogue_sc(row['targets'], row['abs_summary'])['rouge2'][1])
    # word_capt.append(rogue_sc(row['targets'], row['Summary annotated'])['rouge1'][1])
print("------ Beams_summary-------")
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))


------ top_P_abs_summary-------
Average Rouge1 Score:  26.410062309657995
Average Rouge2 Score:  2.0596062026812185
------ Extractive_summary-------
Average Rouge1 Score:  16.323212389587052
Average Rouge2 Score:  1.2723157182016995


KeyError: ignored